The CIFAR-10 dataset that we are using is available directly through the keras module. Therefore, it is easy to download and preprocess. 

In [2]:
# Importing the dataset 
from keras.datasets import cifar10
from keras.utils import np_utils
import matplotlib.pyplot as plt

Below, we create our input (X) and output () matrices for training our neural networks. The keras module already provides separate training and test datasets. 

In [5]:
# loading the data
(train_X, train_Y), (test_X, test_Y) = cifar10.load_data()

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1124)

Now that we have our data, we need to preprocess it. Specifically, we need to convert the pixel values of the images to floats and then normalize the dataset by dividing by the highest pixel value (255).

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
 
train_X = train_X/255.0
test_X = test_X/255.0

Finally, we need to convert our output to categorical variables for us to classify. We do this through one-hot encoding. 

In [ ]:
train_Y = np_utils.to_categorical(train_Y)
test_Y = np_utils.to_categorical(test_Y)
 
num_classes = test_Y.shape[1]